<a href="https://colab.research.google.com/github/jchen42703/CapsNetsLASeg/blob/master/examples/Running_CapsNetsLASeg_Scripts_%5BDemo%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [0]:
!rm -r keras_med_io
!git clone https://github.com/jchen42703/keras_med_io.git
%cd keras_med_io
!pip install .
%cd "/content/"

rm: cannot remove 'keras_med_io': No such file or directory
Cloning into 'keras_med_io'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 894 (delta 36), reused 53 (delta 24), pack-reused 814
Receiving objects: 100% (894/894), 4.70 MiB | 8.02 MiB/s, done.
Resolving deltas: 100% (495/495), done.
/content/keras_med_io
Processing /content/keras_med_io
    100% |████████████████████████████████| 102kB 17.1MB/s 
  Stored in directory: /tmp/pip-ephem-wheel-cache-ofoh7l4g/wheels/8a/be/68/45492440274bf89125626a276a1f68446e94ad69285bae289f
  Stored in directory: /root/.cache/pip/wheels/77/64/60/c6db9959a8f25f106b965893c69c4a853d5b88e55c550fb010
Successfully built keras-med-io batchgenerators


/content


In [0]:
!rm -r CapsNetsLASeg
!git clone https://github.com/jchen42703/CapsNetsLASeg.git
%cd CapsNetsLASeg
!pip install .
%cd "/content/"

Cloning into 'CapsNetsLASeg'...
remote: Enumerating objects: 186, done.
remote: Counting objects: 100% (186/186), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 186 (delta 91), reused 148 (delta 64), pack-reused 0
Receiving objects: 100% (186/186), 11.54 MiB | 13.71 MiB/s, done.
Resolving deltas: 100% (91/91), done.
/content/CapsNetsLASeg
Processing /content/CapsNetsLASeg
  Stored in directory: /tmp/pip-ephem-wheel-cache-wll6gxzx/wheels/7f/e9/36/864269a056f6c4376ca49fbaed6b5fa544410b7e9d0091b1e4
Successfully built capsnets-laseg
  Found existing installation: capsnets-laseg 0.1.1
    Uninstalling capsnets-laseg-0.1.1:
      Successfully uninstalled capsnets-laseg-0.1.1
/content


In [0]:
!ls

CapsNetsLASeg  keras_med_io  sample_data


# Step 1: Downloading and Preprocessing the Dataset

In [0]:
!python /content/CapsNetsLASeg/scripts/download_preprocess.py --dset_path=/content/ --output_path=/content/

Input Directory:  /content/Task02_Heart 
Output Directory:  /content/Preprocessed_Heart
Extracting data [STARTED]
Extracting data [DONE]
Created directory:  /content/Preprocessed_Heart
Metadata scanning completed!
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
Created directory:  /content/Preprocessed_Heart/imagesTr
Created directory:  /content/Preprocessed_Heart/labelsTr
Saved: la_016
Saved: la_014
Saved: la_010
Saved: la_004
Saved: la_030
Saved: la_018
Saved: la_021
Saved: la_029
Saved: la_020
Saved: la_005
Saved: la_023
Saved: la_011
Saved: la_026
Saved: la_003
Saved: la_009
Saved: la_022
Saved: la_024
Saved: la_007
Saved: la_019
Saved: la_017


# Step 2: Training the Network (CNN)

In [0]:
# downloading the fold json
! pip install PyDrive
# saving weights and uploading them to gdrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile, os

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 23.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
# fold 1 json
# https://drive.google.com/open?id= <- id is right here in a share link
fname = "capsnetslaseg_fold1.json"
file_id = "1bT531gJfzOUrX4_iTb2tYUh6PNCywyPr"
print ("Downloading file")
file = drive.CreateFile({'id': file_id})
file.GetContentFile(fname)

In [0]:
!python /content/CapsNetsLASeg/scripts/training.py --weights_dir=/content/ --dset_path=/content/Preprocessed_Heart --model_name=cnn --epochs=2 --batch_size=2 --n_pos=1 --lr=5e-4 --steps_per_epoch=96 --fold_json_path="/content/capsnetslaseg_fold1.json" --max_queue_size=40
# --weights_name="cnn_weights_2epochs.h5" --initial_epoch=2

Using TensorFlow backend.
Using 2D Generators... 
Using at least:  1 positive class slices
Adjusting the indexes since the total number of required samples (steps_per_epoch * batch_size) is greater than the number of provided images.
Done!
Adjusting the indexes since the total number of required samples (steps_per_epoch * batch_size) is greater than the number of provided images.
Done!
Steps per epoch:  96 
Validation Steps:  16
Instructions for updating:
Colocations handled automatically by placer.
Deducing number of pools...
Minimum shape:  [4. 5.]
Number of pools for each corresponding axis:  (6, 6)
Deducing number of pools...
Minimum shape:  [4. 5.]
Number of pools for each corresponding axis:  (6, 6)
2019-04-25 14:39:57.961249: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-04-25 14:39:57.961503: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x1e811e0 executing computations on platform Host. Devices:
2019-04-25 14:39:57.

In [0]:
!ls

adc.json		  cnn_history.pickle	  sample_data
CapsNetsLASeg		  cnn_weights_8epochs.h5  Task02_Heart
capsnetslaseg_fold1.json  keras_med_io		  Task02_Heart.tar
checkpoint.h5		  Preprocessed_Heart


In [0]:
import pickle
with open("cnn_history.pickle", "rb") as input_file:
     history = pickle.load(input_file)
history

{'dice_hard': [0.13654123391703857, 0.4007942648604512],
 'loss': [0.5985863211875161, 0.4266302368293206],
 'lr': [0.0005, 0.0005],
 'val_dice_hard': [0.016809790016850457, 0.011670430729282089],
 'val_loss': [2.864881232380867, 0.731745220720768]}

In [0]:
# uploading weights to gdrive [optional]
folder_id = "..." # <- get id from gdrive share link

# Uploading weights
fname = "cnn_weights_8epochs.h5" # change the fname based on your actual model name and number of epochs
file_dict = {"title": fname, "parents": [{"kind": "drive#fileLink", "id": folder_id}]}
f = drive.CreateFile(file_dict)
f.SetContentFile(fname)
f.Upload()
print("Done uploading!")

# Step 3: Inference (CNN)

In [0]:
!mkdir /content/pred

In [0]:
!python /content/CapsNetsLASeg/scripts/inference.py --weights_path=/content/cnn_weights_2epochs.h5 --raw_dset_path=/content/Task02_Heart --model_name=cnn --fold_json_path="/content/capsnetslaseg_fold1.json" --batch_size=17 --save_dir="/content/pred"

Using TensorFlow backend.
Instructions for updating:
Colocations handled automatically by placer.
Deducing number of pools...
Minimum shape:  [4. 5.]
Number of pools for each corresponding axis:  (6, 6)
Deducing number of pools...
Minimum shape:  [4. 5.]
Number of pools for each corresponding axis:  (6, 6)
2019-04-25 14:53:04.271968: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-04-25 14:53:04.272202: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x20931e0 executing computations on platform Host. Devices:
2019-04-25 14:53:04.272257: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-04-25 14:53:04.462254: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-04-25 14:53:04.462801: I tensorflow/compiler/xla/service/service.cc:15

# Step 4: Visualization

In [0]:
! pip install celluloid

In [0]:
from matplotlib.colors import colorConverter
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
def create_segmentation_cmap(color = "red"): 
  """
  Inspired by: https://stackoverflow.com/questions/10127284/overlay-imshow-plots-in-matplotlib
  Creates a colormap that is transparent besides one color that you specify
  """
  # generate the colors for your colormap
  color1 = colorConverter.to_rgba('white')
  color2 = colorConverter.to_rgba(color)
  # make the colormaps
  seg_cmap = mpl.colors.LinearSegmentedColormap.from_list('my_cmap2',[color1,color2],256)

  seg_cmap._init() # create the _lut array, with rgba values

  # create your alpha array and fill the colormap with them.
  # here it is progressive, but you can create whathever you want
  alphas = np.linspace(0, 0.8, seg_cmap.N+3)
  seg_cmap._lut[:,-1] = alphas
  return seg_cmap



In [0]:
pred = np.load("/content/pred/pred_seg.npy").squeeze()
# y = np.expand_dims(np.load("/content/pred/actual_test_y.npy"), -1)
# x = np.expand_dims(np.load("/content/pred/actual_test_x.npy"), -1)
y = np.load("/content/pred/actual_test_y.npy")
x = np.load("/content/pred/actual_test_x.npy")

In [0]:
pred.shape, y.shape, x.shape

((450, 320, 320), (450, 320, 320), (450, 320, 320))

In [0]:
%matplotlib notebook
from IPython.display import HTML
# animation = camera.animate()
# HTML(animation.to_html5_video())

import numpy as np
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
from matplotlib.colors import hsv_to_rgb
from tqdm import tqdm
from celluloid import Camera

fig = plt.figure()
# hack to remove border
# https://stackoverflow.com/a/37810568/744520
fig.set_size_inches(4, 4, forward=False)
ax = plt.Axes(fig, [0, 0, 1, 1])
ax.set_axis_off()
fig.add_axes(ax)
camera = Camera(fig)
# creates cmap for overlay
cmap_seg = create_segmentation_cmap("red")

desired_img = x
for _slice in range(desired_img.shape[0]):
  img = desired_img[_slice].squeeze()
  label = y[_slice].squeeze()
  ax.imshow(img, cmap = "bone")
  ax.imshow(label, cmap = cmap_seg)
  camera.snap()

animation = camera.animate(interval=200, blit=True)
HTML(animation.to_html5_video())
